One approach to automatic differentiation arises from imposing a commutative ring structure on function and its derivative.

Specifically, we define an algebraic structure on tuples of real-numbers as follows
$$
  (a_0, a_1) + (b_0, b_1) = (a_0 + b_0, a_1 + b_1)
$$
and 
$$
  (a_0, a_1) \cdot (b_0, b_1) = (a_0 \cdot b_0, a_1 \cdot b_0 + a_0 \cdot b_1)
$$

It is straightforwad to verify that this gives defines a commutiative ring with identity $(1, 0)$.   

For example, the above structure is isomorphic to $\mathbb{R}[X]/(X^2).$  

If you don't know what that means, you can think of our approach as treating functions as if they were just formal Taylor series, performing algebraic operations like addition and multiplication on said Taylor series, defining the function via the zeroth term, defining the dervative as the first term, and throwing away all the higher terms.


It is also easy to verify that any element with $a_0 \neq 0$ has a multiplicative inverse, namely
$$
    (a_0, a_1)^{-1} = (1/a_0, -a_1/a_0^2)
$$

## Dual Numbers

Given such a tuple, we'll associate the first component to a variable, say $x$, and the second
component to the differential $dx$.  

We'll refer to $(x, dx)$ tuples, equipped with this algebraic structeure, as *dual numbers*

Rewriting the addition and multiplication rules above recover the familiar sum and product rules from freshman calculus.

$$
  (x, dx) + (y, dy) = (x + y, dx + dy)
$$
and 
$$
  (x, dx) \cdot (y, dy) = (x \cdot y, dx \cdot y + x \cdot dy)
$$


These two rules alone are able to recover many basic results from freshman calculus.

For example, using the multiplication rule recovers the power rule:
$$
    (x, dx) \cdot (x, dx) = (x^2, 2\cdot x\cdot dx)
$$

Combining the multiplication rule with the multiplicative inverse recovers the quotient rule:
$$
    (x, dx) \cdot (y, dy)^{-1}
    = (x, dx) \cdot \left(\frac{1}{y}, \frac{-dy}{y^2}\right)
    = \left(\frac{x}{y}, \frac{1}{y} dx - \frac{x}{y^2} dy\right)
    = \left(\frac{x}{y}, \frac{y \cdot dx - x\cdot dy}{y^2}\right)
$$

## Dual Numbers via Operator Overloading in Python 

In addition to it's theoretical beauty, automatic differentiation with dual numbers
also gives us an excuse to more deeply explore operator overloading in python.  

I had been aware that one could overload the `+`, `*`, `/`, `**` operators for 
your own custom classes, but I had never had a practical excuse for doing so.  

In [1]:
class DualNumber:
    
    def __init__(self, x, dx):
        self.x = x
        self.dx = dx

    def __repr__(self):
        return "{} + {} eps".format(self.x, self.dx)
        
    def __pow__(self, power):
        x = self.x ** power
        dx = power * self.x ** (power - 1) * self.dx
        return self.__class__(x=x, dx=dx)
        
    def __add__(self, other):
        x = self.x + other.x
        dx = self.dx + other.dx
        return self.__class__(x=x, dx=dx)
    
    def __sub__(self, other):
        x = self.x - other.x
        dx = self.dx - other.dx
        return self.__class__(x=x, dx=dx)
    
    def __mul__(self, other):
        x = self.x * other.x
        dx = self.dx * other.x + self.x * other.dx
        return self.__class__(x=x, dx=dx)
    
    def __truediv__(self, other):
        x = self.x / other.x
        dx = (self.dx * other.x - self.x * other.dx) / other.x ** 2
        return self.__class__(x=x, dx=dx)


dual_1 = DualNumber(5, 1)
dual_2 = DualNumber(3, -3)

print(dual_1**3)
print(dual_1 + dual_2)
print(dual_1 * dual_2)
print(dual_1 / dual_2)

125 + 75 eps
8 + -2 eps
15 + -12 eps
1.6666666666666667 + 2.0 eps


See `automatic_diff.dual_number.py` for this idea taken to completion.  

## Functions of Dual Numbers

Let $f:\mathbb{R} \rightarrow \mathbb{R}$.   We need to define a rule that will allow us to extend $f$ to a mapping from dual number to dual number.  (We'll abuse notation, using $f$ to denote either the real-valued function or the dual-number valued function.)
$$
    f((x, dx)) = (f(x),  f'(x)\cdot dx)
$$

### Chain rule?

Suppose we have real valued functions $f$ and $g$ and then define $h = g \circ f$.   
Lets use the above definition to extend $f$ and $g$ to dual-number functions,  
$f((x, dx)) = (f(x), f'(x) \cdot dx)$ and $g((y, dy)) = (g(y), g'(y) \cdot dy)$

Substituting $(f(x), f'(x) \cdot dx)$ for $(y, dy)$ gives 
$$
g(f(x, dx)) = (g(f(x)), g'(f(x))\cdot f'(x)\cdot dx)
$$
In other words, in the world of dual numbers, the chain rule is just a trivial algebraic consequence of how we define functions.


Notice how beautiful this idea is!  In freshman calculus, we think of the functions as somehow being fundamental objects, but derivatives are obtained by performing some tedious action 
on the fundamental function.  In the world of dual numbers, the derivatives are on equal footing with the fundamental function.  The derivatives don't require any extra effort.  They are just, well, automatically there!

Also note that we have not done any calculus here.  No difference quotients.  No limits.   All we've done is to define three algebraic operations, and bunch of basic calculus just falls out of those algebraic operations -- albeit those definitions were directly inspired by calculus theorems.


## What about Sine?

What about $f(x) = \sin{(x)}?$  Unfortunately, the rules above are purely algebraic, but the trigonometric functions are transcendental (fancy way of saying no finite combinations of multiplications, additions, powers, compositions of $x$ will give rise to $\sin{(x)}$).  

In other words, our automatic differentiation system needs our help.  We need to tell it how to extend $\sin$ to a dual-number function.

$$
    \sin{((x, dx))} = (\sin{(x)}, \cos{(x)} \cdot dx)
$$


But now that we've given it that hint, it can take care of the other trig functions on its own, and it can take can over any algebraic combination $x$ and trig functions.

For example, 
$$
   f(x) = x \sin{(x)}
$$
then 
$$
   f((x, dx)) = (x, dx) \cdot (\sin{(x)}, \cos{(x)}\cdot dx)
   = (x\sin{(x)}, dx \cdot \sin{(x)} + x \cos{(x)}\cdot dx )
$$


Technically, the automatic differentiation system should now know how to handle all of the
the other trig functions, since they can all be expressed in terms of $\sin$.  However, in our implementation, we chose to also tell it 
$$
    \cos{((x, dx))} = (\cos{(x)}, -\sin{(x)} \cdot dx)
$$
since that seemed more natural than telling it that $\cos{(x)} = \sin{(\pi/2 - x)}$

In [2]:
import numpy as np

def sin(d: DualNumber):
    return DualNumber(np.sin(d.x), np.cos(d.x) * d.dx)

def cos(d: DualNumber):
    return DualNumber(np.cos(d.x), - np.sin(d.x) * d.dx)

trig_dual = DualNumber(np.pi/6, 1)
tan = lambda d: sin(d)/cos(d)

print(sin(trig_dual))
print(tan(trig_dual))

0.49999999999999994 + 0.8660254037844387 eps
0.5773502691896256 + 1.333333333333333 eps


## Other functions?

Does our automatic differentiation system need any other hints from us?  Unless we bring in complex numbers, the exponentials and logarithms are not expressable in terms of any of the operations we've discussed so far, so we'll need to give it another hint.

$$
    \exp{((x, dx))} = (\exp{(x)}, \exp{(x)} \cdot dx)
$$

Technically, the automatic differentiation system should now know how to handle logarithms as
well, via inverse function rules, but we found it easier to just tell it 
$$
    \ln{(x, dx)} = (\ln{(x)}, dx/x)
$$


This still doesn't cover everything, for example Bessel functions and other similar special functions can't be expressed as finite algebraic combinations of $x$, $trig(x)$, $exp(x)$, $log(x)$.  So if we need to handle Bessel functions, we would need to give our automatic differentiation systems hints on how to deal with a few of them.  My suspicion is that you can never give the system enough hints to allow it to handle *every* differentiable function, but I did not find any references for that claim.  (About 2 minutes of Googling "transcendental degree of differentiable functions" was unsuccessful.
